<a href="https://colab.research.google.com/github/kar-elizabeth/EQUIPAJE/blob/main/08_08_2025_con_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Montar Google Drive para acceder al archivo, si está ahí (opcional)
# from google.colab import drive
# drive.mount('/content/drive')

# Ruta al archivo (ajusta si está en Drive u otra ruta)
archivo_excel = 'DIV_ORIGEN_SEPARADO hoja por hoja.xlsx'

# Leer todas las hojas del archivo Excel
xls = pd.ExcelFile(archivo_excel)
hojas = xls.sheet_names

# Recorrer cada hoja
for hoja in hojas:
    df = pd.read_excel(xls, sheet_name=hoja)

    # Filtrar filas donde la columna "CATEGORIA" sea "Negativo"
    df_negativo = df[df['CATEGORIA'] == 'Negativo']

    # Ranking descendente por frecuencia de "CODIGO_ORIGEN"
    ranking = df_negativo['CODIGO_ORIGEN'].value_counts()

    print(f'Ranking de CODIGO_ORIGEN en hoja "{hoja}" para categoría Negativo:')
    print(ranking)
    print('\n' + '-'*50 + '\n')


Ranking de CODIGO_ORIGEN en hoja "Guadalajara" para categoría Negativo:
CODIGO_ORIGEN
GUAD     465
TZAP      98
COLI      63
MANZ      55
MAZAT     54
PTO.V     46
TTLAQ     30
GUZ       21
AHUT      17
MELA      15
PSOL      14
SAHU      10
MEZCA      8
C.C.       5
ARAN       5
TECO       4
NVOV       4
CIHUA      3
HUER       3
BAR        3
GUAYA      3
ATOT       3
GUAEJ      3
OCO        2
GUADC      1
U.TUL      1
SAYUL      1
TOMA       1
JIQI       1
Name: count, dtype: int64

--------------------------------------------------

Ranking de CODIGO_ORIGEN en hoja "México" para categoría Negativo:
CODIGO_ORIGEN
MEX      779
COAPA    101
PSTFE     64
AEMEX     61
ANGMX     42
MEXOB     34
AMEX2     33
TEPO      24
PLACJ     23
TLANE     21
P.AME     18
AIFA1     15
HCBCA      6
Name: count, dtype: int64

--------------------------------------------------

Ranking de CODIGO_ORIGEN en hoja "Irapuato" para categoría Negativo:
CODIGO_ORIGEN
RI      134
ZAM      55
PIED     33
SALA     2

entrega del analisis en excel. para función simple de datos

In [ ]:
# Instalar xlsxwriter para guardar archivos Excel con pandas
!pip install xlsxwriter

import pandas as pd

archivo_excel = 'DIV_ORIGEN_SEPARADO hoja por hoja.xlsx'
xls = pd.ExcelFile(archivo_excel)
hojas = xls.sheet_names

rankings_por_hoja = {}

for hoja in hojas:
    df = pd.read_excel(xls, sheet_name=hoja)
    df_negativo = df[df['CATEGORIA'] == 'Negativo']
    ranking = df_negativo['CODIGO_ORIGEN'].value_counts().reset_index()
    ranking.columns = ['CODIGO_ORIGEN', 'Frecuencia']
    rankings_por_hoja[hoja] = ranking

archivo_salida = 'ranking_categoria_negativo.xlsx'
with pd.ExcelWriter(archivo_salida, engine='xlsxwriter') as writer:
    for hoja, ranking_df in rankings_por_hoja.items():
        ranking_df.to_excel(writer, sheet_name=hoja[:31], index=False)

print(f'Archivo guardado como: {archivo_salida}')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 4.5 MB/s eta 0:00:00
Archivo guardado como: ranking_categoria_negativo.xlsx


In [ ]:
import pandas as pd

archivo_excel = 'DIV_ORIGEN_SEPARADO hoja por hoja.xlsx'
xls = pd.ExcelFile(archivo_excel)
hojas = xls.sheet_names

archivo_salida = 'ranking_y_detalle_categoria_negativo.xlsx'

with pd.ExcelWriter(archivo_salida, engine='xlsxwriter') as writer:
    for hoja in hojas:
        df = pd.read_excel(xls, sheet_name=hoja)

        # Filtrar filas con categoria 'Negativo' conservando todas las columnas
        df_negativo = df[df['CATEGORIA'] == 'Negativo']

        # Ranking de CODIGO_ORIGEN
        ranking = df_negativo['CODIGO_ORIGEN'].value_counts().reset_index()
        ranking.columns = ['CODIGO_ORIGEN', 'Frecuencia']

        # Guardar detalle completo en hoja con nombre original + '_detalle' (limitar a 31 caracteres)
        detalle_hoja = (hoja[:24] + '_detalle') if len(hoja) > 24 else hoja + '_detalle'
        df_negativo.to_excel(writer, sheet_name=detalle_hoja, index=False)

        # Guardar ranking en hoja con nombre original + '_ranking'
        ranking_hoja = (hoja[:24] + '_ranking') if len(hoja) > 24 else hoja + '_ranking'
        ranking.to_excel(writer, sheet_name=ranking_hoja, index=False)

print(f'Archivo guardado con detalle y ranking: {archivo_salida}')


Archivo guardado con detalle y ranking: ranking_y_detalle_categoria_negativo.xlsx


In [ ]:
import pandas as pd

archivo_excel = 'DIV_ORIGEN_SEPARADO hoja por hoja.xlsx'
xls = pd.ExcelFile(archivo_excel)
hojas = xls.sheet_names

archivo_salida = 'ranking_y_detalle_categoria_negativo_ordenado.xlsx'

with pd.ExcelWriter(archivo_salida, engine='xlsxwriter') as writer:
    for hoja in hojas:
        df = pd.read_excel(xls, sheet_name=hoja)

        # Filtrar filas con categoria 'Negativo'
        df_negativo = df[df['CATEGORIA'] == 'Negativo']

        # Ranking de CODIGO_ORIGEN
        ranking = df_negativo['CODIGO_ORIGEN'].value_counts().reset_index()
        ranking.columns = ['CODIGO_ORIGEN', 'Frecuencia']

        # Crear diccionario para ordenamiento: CODIGO_ORIGEN -> rango (menor = más frecuente)
        orden_codigos = {codigo: i for i, codigo in enumerate(ranking['CODIGO_ORIGEN'])}

        # Agregar columna temporal para ordenar
        df_negativo['OrdenRanking'] = df_negativo['CODIGO_ORIGEN'].map(orden_codigos)

        # Ordenar por OrdenRanking y luego opcionalmente por otra columna (por ejemplo, índice)
        df_negativo_ordenado = df_negativo.sort_values(by=['OrdenRanking']).drop(columns=['OrdenRanking'])

        # Guardar detalle ordenado
        detalle_hoja = (hoja[:24] + '_detalle') if len(hoja) > 24 else hoja + '_detalle'
        df_negativo_ordenado.to_excel(writer, sheet_name=detalle_hoja, index=False)

        # Guardar ranking
        ranking_hoja = (hoja[:24] + '_ranking') if len(hoja) > 24 else hoja + '_ranking'
        ranking.to_excel(writer, sheet_name=ranking_hoja, index=False)

print(f'Archivo guardado con detalle ordenado y ranking: {archivo_salida}')


/tmp/ipython-input-2134046479.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_negativo['OrdenRanking'] = df_negativo['CODIGO_ORIGEN'].map(orden_codigos)
/tmp/ipython-input-2134046479.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_negativo['OrdenRanking'] = df_negativo['CODIGO_ORIGEN'].map(orden_codigos)
/tmp/ipython-input-2134046479.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

Archivo guardado con detalle ordenado y ranking: ranking_y_detalle_categoria_negativo_ordenado.xlsx
